In this notebook, we're going to talk about the importance of the spacing between data points in a time series and how to adjust that spacing post-hoc in software.


# Important: Run this code cell each time you start a new session!

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install ipywidgets
!pip install os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
!wget -Ncnp https://physionet.org/files/accelerometry-walk-climb-drive/1.0.0/raw_accelerometry_data/id00b70b13.csv

In [ ]:
df = pd.read_csv('id00b70b13.csv')

# Filter to only walking activity, which is given a code of 1
df = df[df['activity'] == 1]

# Process the time
df.rename(columns={'time_s': 'Time'}, inplace=True)
df = df[(df['Time']>=700) & (df['Time']<=710)]
df['Time'] = df['Time'] - df['Time'].min()

# Process the accel
df['Accel'] = np.sqrt(df['la_x']**2 + df['la_y']**2 + df['la_z']**2)*9.8

# Keep only crucial columns
keep_cols = ['Time', 'Accel']
df = df[keep_cols]
df.to_csv('walking.csv',index=False)

# What Is Sampling Rate?

Most phonemona that we would want to represent using time-series data is inherently continuous, but capturing that information into a digital signal that can be processed requires taking mesaurements at discrete time points.

For example, there are two ways that we could record the outdoor air temperature throughout the day:
1. We could periodically look at a thermometer and write down the readings or, in which case we could maybe measure the temperature every minute or so
2. We could program a temperature sensor to periodically detect, transmit, and store the sensed temperature, in which case we could probably get hundreds of measurements per second.

Regardless, there is no way of feasibly recording the temperature at every possible moment that theoretically exists.

The ***sampling rate*** is the frequency at which a continuous signal is sampled or measured in order to convert it into a digital signal. The units that we typical use to discuss sampling rate is samples per second, also known as Hertz (Hz).

The sampling rate determines the level of detail and accuracy of the digital representation of the original signal. A higher sampling rate results in a more accurate representation of the original signal, but it also requires more storage and computational power to process.

To understand the implications of a sampling rate, let's digitally sample a signal that we will pretend to be a continuous sinusoid.

In the code cell below, the black line represents the continuous signal, while the red points indicate the digital samples that are recorded according to the sampling rate. Bear in mind that the red lines are strictly used to show the shape of the data being summarized. In actuality, the only information that is being saved is the red circles.

Look at the impact of increasing and decreasing the sampling rate by moving the `sampling_rate` slider and then re-running the code block.

In [ ]:
sampling_rate = 10 #@param {type:"slider", min:5, max:25, step:5}

# Calculate the "real signal"
x = np.linspace(0, 1, 1000)
y = np.sin(2*np.pi*x)

# Calculate the "sampled signal"
x_sampled = np.linspace(0, 1, sampling_rate)
y_sampled = np.sin(2*np.pi*x_sampled)

# Plot both of them
plt.figure(figsize=(5, 3))
plt.plot(x, y, 'k-')
plt.plot(x_sampled, y_sampled, 'r-')
plt.stem(x_sampled, y_sampled, 'r')
plt.xlabel('Time (s)')
plt.ylabel('sin(x)')
plt.show()

When our sampling rate is very low, the sampled points do not sufficiently summarize the shape of the smooth curvy signal. As we increase the sampling rate, the lines drawn between the points look closer to our ideal sinuosoid.

# Changing the Sampling Rate

Depending on the signal you are capturing, you may or may not have control over how the data is sampled before you load it in your code. Here are some scenarios where you may have control over the sampling rate of your data:
* You are using a medical device or sensor that allows you to change the setting of how often it produces a measurement
* You have a separate program that periodically queries an API for data (e.g., weather forecasts)
* You are asking people to fill out a questionnaire or survey at a fixed interval

You might find yourself in situations when you will want to digitally change the sampling rate of your data to simplify your analysis:
1. You might want to **upsample**, or increase the sampling rate of your signal, to fill in gaps or provide more fidelity for downstream analysis.
2. You might want to **downsample**, or decrease the sampling rate of your signal, to speed up your code by reducing the amount of data you need to process.
3. You might want to make the sampling **more uniform** because it was not collected at a regular interval.

One way we can modify the sampling rate of our data is by using ***interpolation***. Interpolation refers to the idea of estimating values that lie between two known data points. There are multiple variations of interpolation; some examples are described below in increasing order of complexity:
* **Nearest neighbor interpolation:** Missing values are assigned based on the closest data point from the original signal.
* **Linear interpolation:** Missing values are assigned based on a line that is drawn between the two closest data points from the original signal.
* **Spline interpolation:** Missing values are assigned based on piecewise polynomial functions fit to sections of the original signal.

The `interp()` function in `numpy` is one of the convenient ways of performing linear interpolation. This function requires three inputs:
1. The times at which the new samples should be generated
2. The times of the original samples
3. The values of the original samples

The code cell below illustrates the output of this function. The time series in black represents our initial signal, which is sampled at 20 Hz. The variable `new_sampling_rate` will dictate how many points we want in the new time series between the start and end of the original signal.

Play around with different settings of `new_sampling_rate` to see what happens. As you can tell by the range of the slider, you can actually use this function to upsample or downsample the signal.

In [ ]:
new_sampling_rate = 25 #@param {type:"slider", min:5, max:35, step:5}

# Generate the original signal
x = np.linspace(0, 1, 20)
y = np.sin(2*np.pi*x)
max_x = np.max(x)

# Calculate where the new points should be
x_resampled = np.linspace(0, max_x, int(max_x*new_sampling_rate))

# Use the interp function to calculate where the new y-values should be
y_resampled = np.interp(x_resampled, x, y)

# Plot both of them
plt.figure(figsize=(5, 3))
plt.plot(x, y, 'k-o')
plt.plot(x_resampled, y_resampled, 'rx')
plt.xlabel('Time (s)')
plt.ylabel('sin(x)')
plt.show()

This technique can also work in situations when data is missing, but bear in mind that interpolation can only generate data using information that is already available in the original signal.

In [ ]:
# Generate the original signal
x = np.linspace(0, 1, 20)
y = np.sin(2*np.pi*x)

# Remove some data points
remove_idx = [5, 6, 7, 18]
x = np.delete(x, remove_idx)
y = np.delete(y, remove_idx)

# Calculate where the new points should be
x_resampled = np.linspace(0, 1, 20)

# Use the interp function to calculate where the new y-values should be
y_resampled = np.interp(x_resampled, x, y)

# Plot both of them
plt.figure(figsize=(5, 3))
plt.plot(x, y, 'k-o')
plt.plot(x_resampled, y_resampled, 'rx')
plt.xlabel('Time (s)')
plt.ylabel('sin(x)')
plt.show()